# Gilead - Accumulations
## Next year
* No Accumulations. Correct?

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("Gilead_Sciences_Switzerland_EFPIA_Report_2018.pdf", pages='all', lattice=True, pandas_options={'thousands': '.'})

## Format Table

In [8]:
df_export = df.copy()

#Clear cells with not more than 1 char
column = df_export.columns[0]
df_export.loc[df_export[column].str.len() <= 1, column] = np.NaN

#Shift row
column = df_export.columns[0]
df_export[df_export[column].isna()] = df_export[df_export[column].isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[[7]],axis=1)
df_export = df_export.drop(df.columns[9:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of', na=False)]
rnd = df_export[-1:]

df_export = pd.concat([amount, recipients, rnd])

#Convert to Numbers
#df_export = remove_dots(df_export)
df_export = remove_comma(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#Shift RND from Sponsorship to total
df_export.loc[df_export.type == 'rnd', 'total'] = df_export.donations_grants
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

export_acumulations(df_export, 'gilead')

Total nicht Summe der Werte
saved


In [4]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)